In [1]:
import pandas as pd
import random as r

In [2]:
attendance_raw = []

cohorts = ["alpha", "beta", "gamma"]

for i in range(1, 25):
    record = {
        "student_id": f"S{i:03}",
        "cohort": r.choice(cohorts),
        "attended_sessions": r.randint(0, 6),
        "expected_sessions": 6
    }

    attendance_raw.append(record)

attendance = pd.DataFrame(attendance_raw)
print(attendance.head())
print(attendance.info())

  student_id cohort  attended_sessions  expected_sessions
0       S001  gamma                  1                  6
1       S002  gamma                  1                  6
2       S003   beta                  3                  6
3       S004  gamma                  0                  6
4       S005  gamma                  1                  6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   student_id         24 non-null     object
 1   cohort             24 non-null     object
 2   attended_sessions  24 non-null     int64 
 3   expected_sessions  24 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 900.0+ bytes
None


In [3]:
attendance_indexed = attendance.set_index("student_id")

IDs = []

for i in range(16, 30, 2):
    IDs.append(f"S{i:03}")

excused_absences = pd.Series(IDs)

excused_indexes = pd.Series(1, index = excused_absences)

attendance_indexed["adjusted_attendance"] = (
    attendance_indexed["attended_sessions"] + excused_indexes
)

print("The sum of all the NaN values:", attendance_indexed["adjusted_attendance"].isna().sum(), "\n")

attendance_indexed["adjusted_attendance"] = (
    attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])
)

print("The updated columns:\n", attendance_indexed[["adjusted_attendance", "attended_sessions"]].head()) 

The sum of all the NaN values: 19 

The updated columns:
             adjusted_attendance  attended_sessions
student_id                                        
S001                        1.0                  1
S002                        1.0                  1
S003                        3.0                  3
S004                        0.0                  0
S005                        1.0                  1


In [4]:
attendance_dyrti = attendance_indexed.copy()

attendance_dyrti.loc[attendance_dyrti.index[0], "cohort"] = "gamma    "
attendance_dyrti.loc[attendance_dyrti.index[3], "cohort"] = "gammA"
attendance_dyrti.loc[attendance_dyrti.index[6], "cohort"] = "gAMma    "
attendance_dyrti.loc[attendance_dyrti.index[8], "cohort"] = "gAmma    "

attendance_dyrti["cohort"] = (
    attendance_dyrti["cohort"]
    .str.strip()
    .str.lower()
)

attendance_dyrti["cohort"].unique()

array(['gamma', 'beta', 'alpha'], dtype=object)

In [5]:
low_attendance = attendance_dyrti[attendance_dyrti["attended_sessions"] < attendance_dyrti["expected_sessions"]]

summary = low_attendance.groupby("cohort")["attended_sessions"].mean()
print(summary)

set(summary.index) == set(attendance_dyrti["cohort"].unique())

cohort
alpha    2.428571
beta     2.750000
gamma    1.600000
Name: attended_sessions, dtype: float64


True

In [6]:
attendance_dyrti["attendance_ok"] = (
        attendance_dyrti["attended_sessions"] >= attendance_dyrti["expected_sessions"]
)

low_attendance = attendance_dyrti[attendance_dyrti["attended_sessions"] < attendance_dyrti["expected_sessions"]]

validation = (low_attendance["attendance_ok"] == False).all()
print(validation)

True
